# Agent Development to use multiple Tools
#### Let us create  tools, Wikipedia Search, Webpage Search, Arxiv Search and create an Agent that can use all these tools and give an output based on using the perfet tool for the given input query

### Wikipedia Tool creation

In [2]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

### WebPage Tool Creation (Specifically for LangSmith)

In [6]:
from langchain_community.document_loaders import WebBaseLoader
import bs4
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.langchain.com/langsmith/home")
document = loader.load()
chunks = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(document)
db = FAISS.from_documents(chunks, OllamaEmbeddings(model="mxbai-embed-large"))
retriver = db.as_retriever()


In [8]:
from langchain_core.tools import create_retriever_tool
langsmith_tool = create_retriever_tool(retriver, "langsmith_search", "Search for information about LangSmith. For any information about LangSmith use this tool.")

### Arxiv Tool Development

In [12]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv_tool = ArxivQueryRun(api_erapper=arxiv_wrapper)

In [78]:
all_tools = [wiki_tool, langsmith_tool, arxiv_tool]
all_tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'b:\\Notes\\Projects\\Langchain_Agenti_AI_Dev\\.venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 StructuredTool(name='langsmith_search', description='Search for information about LangSmith. For any information about LangSmith use this tool.', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=<function create_retriever_tool.<locals>.func at 0x000002ECE1398860>, coroutine=<function create_retriever_tool.<locals>.afunc at 0x000002ECE13984A0>),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=3, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=4000))]

## Agent Development

In [79]:
from langchain_ollama.llms import OllamaLLM

llm = OllamaLLM(model="gemma3:1b")

In [80]:
from langchain_classic import hub

prompt = hub.pull("hwchase17/react")
prompt

PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'}, template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')

In [81]:
from langchain_classic.agents import create_react_agent
agent = create_react_agent(llm, all_tools, prompt)

In [82]:
from langchain_classic.agents import AgentExecutor
agent_exe = AgentExecutor(agent=agent, tools=all_tools, verbose=True)

### Based on the query the Agent invokes the most compatible Tool as seen below

In [84]:
agent_exe.invoke({"input":"What is LangSmith? Not LangChain"})



> Entering new AgentExecutor chain...
I need to find information about LangSmith. I will start by searching on Wikipedia.
Action: wikipedia
Action Input: "LangSmith"Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChainI need to understand what LangChain is. Let's start with Wikipedia.
Action: wikipedia
Action Input: "LangChain"Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChainallows developers to build applications that leverage LLMs. It offers tools for data connection, model management, agent development, and more. It’s built on top of PyTorch and focuses on streamlining the process of deploying and managing LLMs. It’s designed to be easy to use and offers a lot of flexibility for different

{'input': 'What is LangSmith? Not LangChain',
 'output': 'LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain allows developers to build applications that leverage LLMs. It offers tools for data connection, model management, agent development, and more. It’s built on top of PyTorch and focuses on streamlining the process of deploying and managing LLMs.'}

In [67]:
agent_exe.invoke({"input":"Tell me about Machine Learning."})



> Entering new AgentExecutor chain...
I need to understand what Machine Learning is. I'll start with a general overview.
Action: wikipedia
Action Input: "Machine learning"Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and genI understand the basic definition of Machine Learning. It’s about algorithms that learn from data without being explicitly programmed.
Final Answer: Machine learning is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to new, unseen data.

> Finished chain.


{'input': 'Tell me about Machine Learning.',
 'output': 'Machine learning is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to new, unseen data.'}

In [69]:
agent_exe.invoke({"input":"Tell me about paper 	arXiv:2211.10527."})



> Entering new AgentExecutor chain...
I need to find out more about the paper with the arXiv identifier 2211.10527. I’ll start by using the `arxiv` tool to search for this paper.
Action: arxiv
Action Input: 2211.10527

b:\Notes\Projects\Langchain_Agenti_AI_Dev\.venv\Lib\site-packages\langchain_community\utilities\arxiv.py:102: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  ).results()


Published: 2023-05-16
Title: PMNet: Robust Pathloss Map Prediction via Supervised Learning
Authors: Ju-Hyung Lee, Omer Gokalp Serbetci, Dheeraj Panneer Selvam, Andreas F. Molisch
Summary: Pathloss prediction is an essential component of wireless network planning. While ray tracing based methods have been successfully used for many years, they require significant computational effort that may become prohibitive with the increased network densification and/or use of higher frequencies in 5G/B5G (beyond 5G) systems. In this paper, we propose and evaluate a data-driven and model-free pathloss prediction method, dubbed PMNet. This method uses a supervised learning approach: training a neural network (NN) with a limited amount of ray tracing (or channel measurement) data and map data and then predicting the pathloss over location with no ray tracing data with a high level of accuracy. Our proposed pathloss map prediction-oriented NN architecture, which is empowered by state-of-the-art comput

b:\Notes\Projects\Langchain_Agenti_AI_Dev\.venv\Lib\site-packages\langchain_community\utilities\arxiv.py:102: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  ).results()


Published: 2023-05-16
Title: PMNet: Robust Pathloss Map Prediction via Supervised Learning
Authors: Ju-Hyung Lee, Omer Gokalp Serbetci, Dheeraj Panneer Selvam, Andreas F. Molisch
Summary: Pathloss prediction is an essential component of wireless network planning. While ray tracing based methods have been successfully used for many years, they require significant computational effort that may become prohibitive with the increased network densification and/or use of higher frequencies in 5G/B5G (beyond 5G) systems. In this paper, we propose and evaluate a data-driven and model-free pathloss prediction method, dubbed PMNet. This method uses a supervised learning approach: training a neural network (NN) with a limited amount of ray tracing (or channel measurement) data and map data and then predicting the pathloss over location with no ray tracing data with a high level of accuracy. Our proposed pathloss map prediction-oriented NN architecture, which is empowered by state-of-the-art comput

b:\Notes\Projects\Langchain_Agenti_AI_Dev\.venv\Lib\site-packages\langchain_community\utilities\arxiv.py:102: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  ).results()


Published: 2023-05-16
Title: PMNet: Robust Pathloss Map Prediction via Supervised Learning
Authors: Ju-Hyung Lee, Omer Gokalp Serbetci, Dheeraj Panneer Selvam, Andreas F. Molisch
Summary: Pathloss prediction is an essential component of wireless network planning. While ray tracing based methods have been successfully used for many years, they require significant computational effort that may become prohibitive with the increased network densification and/or use of higher frequencies in 5G/B5G (beyond 5G) systems. In this paper, we propose and evaluate a data-driven and model-free pathloss prediction method, dubbed PMNet. This method uses a supervised learning approach: training a neural network (NN) with a limited amount of ray tracing (or channel measurement) data and map data and then predicting the pathloss over location with no ray tracing data with a high level of accuracy. Our proposed pathloss map prediction-oriented NN architecture, which is empowered by state-of-the-art comput

{'input': 'Tell me about paper \tarXiv:2211.10527.',
 'output': 'The paper is titled PMNet: Robust Pathloss Map Prediction via Supervised Learning, authored by Ju-Hyung Lee, Omer Gokalp Serbetci, Dheeraj Panneer Selvam, and Andreas F. Molisch. It was published in 2023 by arXiv:2211.10527. The paper describes a supervised learning pathloss prediction method called PMNet that outperforms other architectures using a limited amount of data and achieves high accuracy.'}